In [ ]:
import pandas as pd
import requests

# URL pour les données des communes de l'INSEE
url = "https://www.insee.fr/fr/statistiques/fichier/6800675/v_commune_2023.csv"

# Charger le fichier CSV contenant la liste des communes
df = pd.read_csv(url, delimiter=";")  # Le séparateur est probablement ';' pour les fichiers français
print("Fichier chargé avec succès !")
print(df.head())

# URL de base pour l'API Geo-DVF
geodvf_api_url = "https://files.data.gouv.fr/geo-dvf/latest/"

# Fonction pour appeler l'API Geo-DVF et obtenir les données de prix pour une commune
def get_price_data(commune_code):
    request_url = f"{geodvf_api_url}?code_commune={commune_code}&page=1&page_size=10"
    response = requests.get(request_url)
    if response.status_code == 200:
        json_data = response.json()
        if json_data and 'data' in json_data and len(json_data['data']) > 0:
            price_data = pd.DataFrame(json_data['data'])
            return price_data
    return None

# Liste pour stocker les DataFrames individuels
all_price_data = []

# Boucle pour récupérer les données pour chaque commune
for index, row in df.iterrows():
    commune_code = row['COM']  # Remplacez par la colonne contenant le code INSEE si différent
    price_data = get_price_data(commune_code)
    if price_data is not None:
        all_price_data.append(price_data)

# Combiner tous les DataFrames en un seul
if all_price_data:
    combined_price_data = pd.concat(all_price_data, ignore_index=True)
    print("Données combinées des prix :")
    print(combined_price_data.head())
    # Enregistrer dans un fichier CSV
    combined_price_data.to_csv("price_data.csv", index=False)
    print("Données sauvegardées dans 'price_data.csv'")
else:
    print("Aucune donnée valide trouvée pour les communes.")
